# Qwen2-VL-7B-Instruct
Qwen/Qwen2-VL-7B-Instruct

In [ ]:
# ! pip install bitsandbytes peft trl
# ! pip install trl

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"  # weights and boases comes enabled by default

In [ ]:
from datasets import load_dataset
import torch
from transformers import (
    Qwen2VLForConditionalGeneration,
    Qwen2VLProcessor,
    BitsAndBytesConfig,  # quatinzation
)


from peft import (
    LoraConfig,
    get_peft_model,
)  # as we dont have enough gpu to work with full vision model so we are gpoing with adapaters such as LoRa


from trl import SFTConfig, SFTTrainer


import warnings

warnings.filterwarnings("ignore")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"{device=}")


MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"

EPOCHS = 1
BATCH_SIZE = 1


GRADIENT_CHECKPOINTING = True
USE_REENTRANT = False
OPTIM = "paged_adamw_32bit"
LEARNING_RATE = 2e-5
LOGGING_STEPS = 50
EVAL_STEPS = 50
SAVE_STEPS = 50
SAVE_STRATEGY = "steps"
EVAL_STRATEGY = "steps"
METRIC_FOR_BEST_MODEL = (
    "eval_loss"  # to get better model which might not be the last saved model
)
LOAD_BEST_MODEL_AT_END = True
MAX_GRAD_NORM = 1
WARMUP_STEPS = 0
DATASET_KWARGS = {"skip_prepare_dataset": True}  # we have to put for VLMs

REMOVE_UNUSED_COLUMNS = False  # VLM thing
MAX_SEQ_LEN = 128  # max seq len of the generated text
DATA_POINTS_IN_DATASET = 283
NUM_STEPS = (DATA_POINTS_IN_DATASET // BATCH_SIZE) * EPOCHS
print(f"{NUM_STEPS=}")

device='cuda:0'
NUM_STEPS=283


In [ ]:
# our data will need to have format that is compatible with VLM and SIFT trainer
# so for every data point in dataset we will the points in a fomrat that we want
# for that first we'll display a system_message

system_message = """ You are a highly advanced Vision Language Model (VLM), specializing in analyzing, describing and interpreting visual data.
Your task is to process and extract meaningful insights from images, videos and visual patterns,
leveraging multimodal understanding to provide accurate and contextually relevant infomration."""
print(f"{system_message=}")


def format_data(sample):
    return [
        {"role": "system", "content": [{"type": "text", "text": system_message}]},
        {
            "role": "user",
            "content": [
                {"type": "image", "image": sample["image"]},
                {"type": "text", "text": sample["query"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["label"][0]}],
        },
    ]

system_message=' You are a highly advanced Vision Language Model (VLM), specializing in analyzing, describing and interpreting visual data.\nYour task is to process and extract meaningful insights from images, videos and visual patterns,\nleveraging multimodal understanding to provide accurate and contextually relevant infomration.'


In [ ]:
train_dataset, eval_dataset, test_dataset = load_dataset(
    "HuggingFaceM4/ChartQA",
    split=["train[:1%]", "val[:1%]", "test[:1%]"],  # only using 1% of data
)

README.md:   0%|          | 0.00/852 [00:00<?, ?B/s]

(…)-00000-of-00003-49492f364babfa44.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

(…)-00001-of-00003-7302bae5e425bbc7.parquet:   0%|          | 0.00/311M [00:00<?, ?B/s]

(…)-00002-of-00003-194c9400785577a2.parquet:   0%|          | 0.00/315M [00:00<?, ?B/s]

(…)-00000-of-00001-0f11003c77497969.parquet:   0%|          | 0.00/50.2M [00:00<?, ?B/s]

(…)-00000-of-00001-e2cd0b7a0f9eb20d.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28299 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1920 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
train_dataset, eval_dataset